In [1]:
import SimpleITK as sitk
from glob import glob
from pathlib import Path
import json
import pandas as pd
import os
import tqdm
import numpy as np

In [2]:
train_im_dir = "/work/vig/Datasets/aneurysm/internal_train/og_0.4"
test_im_dir = "/work/vig/Datasets/aneurysm/internal_test_0.4"
train_mask_dir = "/work/vig/Datasets/aneurysm/internal_train/og_0.4_label"
test_mask_dir = "/work/vig/Datasets/aneurysm/internal_test/og_0.4_label"

train_im_dest = Path("/work/vig/Datasets/aneurysm/internal_train/crop_0.4")
test_im_dest = Path("/work/vig/Datasets/aneurysm/internal_test/crop_0.4")
train_mask_dest = Path("/work/vig/Datasets/aneurysm/internal_train/crop_0.4_label")
test_mask_dest = Path("/work/vig/Datasets/aneurysm/internal_test/crop_0.4_label")

path_offsets_train_scan =  "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_train_0.4_crop.json"
path_offsets_test_scan =  "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_test_0.4_crop.json"
path_offsets_train_label =  "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_label_train_0.4_crop.json"
path_offsets_test_label =  "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_label_test_0.4_crop.json"

train_im_files = sorted(list(glob(f"{train_im_dir}/*")))
val_im_files = sorted(list(glob(f"{test_im_dir}/*")))
train_mask_files = sorted(list(glob(f"{train_mask_dir}/*")))
test_mask_files = sorted(list(glob(f"{test_mask_dir}/*")))
len(train_im_files), len(val_im_files), len(train_mask_files), len(test_mask_files)

(1186, 152, 1186, 152)

In [3]:
crop_threshold_train = 425
crop_threshold_test = 425
crop_threshold_with_neck = 700
crop_padding = 15

In [18]:
part = "test"
ver = "label" # scan, label
if part == "test":
    if ver == "label":
        files = test_mask_files
        dest_dir = test_mask_dest
        crop_threshold = crop_threshold_test
        path_offsets = path_offsets_test_label
    elif ver == "scan":
        files = val_im_files
        dest_dir = test_im_dest
        crop_threshold = crop_threshold_test
        path_offsets = path_offsets_test_scan
elif part == "train":
    if ver == "label":
        files = train_mask_files
        dest_dir = train_mask_dest
        crop_threshold = crop_threshold_train
        path_offsets = path_offsets_train_label
    elif ver == "scan":
        files = train_im_files
        dest_dir = train_im_dest
        crop_threshold = crop_threshold_train
        path_offsets = path_offsets_train_scan


In [20]:
combs = [("train", "scan"), ("train", "label"), ("test", "scan"), ("test", "label")]

for part, ver in combs:
    print(f"Doing {part}, {ver}")
    # define where to put files
    if part == "test":
        if ver == "label":
            files = test_mask_files
            dest_dir = test_mask_dest
            crop_threshold = crop_threshold_test
            path_offsets = path_offsets_test_label
        elif ver == "scan":
            files = val_im_files
            dest_dir = test_im_dest
            crop_threshold = crop_threshold_test
            path_offsets = path_offsets_test_scan
    elif part == "train":
        if ver == "label":
            files = train_mask_files
            dest_dir = train_mask_dest
            crop_threshold = crop_threshold_train
            path_offsets = path_offsets_train_label
        elif ver == "scan":
            files = train_im_files
            dest_dir = train_im_dest
            crop_threshold = crop_threshold_train
            path_offsets = path_offsets_train_scan

    dict_crop_offsets = {}
    for idx in range(len(files)):
        print(idx)
        scan = sitk.ReadImage(files[idx])
        array = sitk.GetArrayFromImage(scan)
        scan_seriesuid = files[idx].split("/")[-1]
        if array.shape[0] > crop_threshold:
            crop_point = array.shape[0] - crop_threshold
            # new_array = array[crop_point:, :, :]
            crop = sitk.CropImageFilter()
            crop.SetLowerBoundaryCropSize([0, 0, crop_point])
            cropped_scan = crop.Execute(scan)
            dict_crop_offsets[scan_seriesuid] = crop_point
        else:
            cropped_scan = scan
            dict_crop_offsets[scan_seriesuid] = 0

        # save scan

        path_cropped_scan = dest_dir / scan_seriesuid
        sitk.WriteImage(cropped_scan, path_cropped_scan)
    print("Writing offsets...\n\n\n")
    with open(
        path_offsets,
        "w",
    ) as f:
        json.dump(dict_crop_offsets, f)

Doing train, scan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


In [32]:
with open(
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_test_0.4_crop.json",
    "w",
) as f:
    json.dump(dict_crop_offsets, f)

# Fixing up CSV files


In [2]:
path_offsets_train = "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_train_0.4_crop.json"
path_offsets_test = "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/scan_processing/offsets_internal_test_0.4_crop.json"

In [3]:
import pandas as pd

df_offsets_train = pd.read_json(path_offsets_train, orient="index").reset_index()
df_offsets_test = pd.read_json(path_offsets_test, orient="index").reset_index()

In [9]:
label_file_train = "/work/vig/Datasets/aneurysm/train0.4.csv"
label_file_test = "/work/vig/Datasets/aneurysm/test0.4.csv"

df_train = pd.read_csv(label_file_train)
df_test = pd.read_csv(label_file_test)

In [8]:
keep_columns = ["seriesuid", "coordX", "coordY", "coordZ", "w", "h", "d", "lesion"]

df_train_crop = df_train.merge(df_offsets_train, left_on="seriesuid", right_on="index")
df_train_crop["coordZ"] = df_train_crop["coordZ"] - df_train_crop[0]
df_train_crop["coordZ"] = df_train_crop["coordZ"].apply(lambda x: x if x >= crop_padding or x < 0 else -1)
df_train_crop = df_train_crop[df_train_crop["coordZ"] > 0]
df_train_crop = df_train_crop[keep_columns]

df_test_crop = df_test.merge(df_offsets_test, left_on="seriesuid", right_on="index")
df_test_crop["coordZ"] = df_test_crop["coordZ"] - df_test_crop[0]
df_test_crop = df_test_crop[keep_columns]


In [ ]:
len(df_train), len(df_train_crop), len(df_test), len(df_test_crop), f"Removed {len(df_train) - len(df_train_crop)}"

(1391, 1359, 126, 126, 'Removed 32')

In [66]:
for i in range(1, 1187):
    uid = f"Tr{str(i).zfill(4)}.nii.gz"
    total_in_scan = len(df_train[df_train["seriesuid"] == uid])
    if total_in_scan == 0:
        print(uid, total_in_scan)

In [58]:
label_file_train_dest = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/gt/internal_train_crop_0.4.csv"
label_file_test_dest = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/internal_test_crop_0.4.csv"

df_train_crop.to_csv(label_file_train_dest, index=False)
df_test_crop.to_csv(label_file_test_dest, index=False)


In [52]:
df_train_crop.sort_values("coordZ").head(10)

,seriesuid,coordX,coordY,coordZ,w,h,d,lesion,index,0
861,Tr0734.nii.gz,396.0,267.0,20.0,0.4,0.4,0.4,aneurysm,Tr0734.nii.gz,475
862,Tr0734.nii.gz,409.0,267.0,22.0,0.4,0.4,0.4,aneurysm,Tr0734.nii.gz,475
945,Tr0794.nii.gz,236.5,227.5,25.5,8.0,6.4,7.2,aneurysm,Tr0794.nii.gz,0
374,Tr0326.nii.gz,412.5,426.5,30.5,0.8,7.2,4.8,aneurysm,Tr0326.nii.gz,441
547,Tr0483.nii.gz,286.5,236.5,30.5,6.4,4.8,7.2,aneurysm,Tr0483.nii.gz,0
212,Tr0178.nii.gz,157.5,224.5,36.0,8.8,8.8,13.2,aneurysm,Tr0178.nii.gz,494
1327,Tr1129.nii.gz,151.0,219.0,37.0,0.4,0.4,0.4,aneurysm,Tr1129.nii.gz,0
1143,Tr0962.nii.gz,266.0,312.0,40.5,5.2,6.0,3.2,aneurysm,Tr0962.nii.gz,0
1326,Tr1129.nii.gz,164.5,220.5,40.5,15.2,15.2,19.2,aneurysm,Tr1129.nii.gz,0
1148,Tr0966.nii.gz,298.5,218.5,42.5,4.0,4.8,4.0,aneurysm,Tr0966.nii.gz,0


# Create METADATA file


In [4]:
# make json
meta = {}

path_files = Path(train_im_dest)
for file in tqdm.tqdm(path_files.glob("*.nii.gz")):
    header = sitk.ReadImage(file)
    idx = os.path.split(file)[1]
    meta[idx] = {
        "origin": list(header.GetOrigin()),
        "spacing": list(header.GetSpacing()),
    }

# path

path_json = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/internal_train_crop_meta.json"
with open(path_json, "w") as f:
    json.dump(meta, f)

1186it [27:32,  1.39s/it]


# External datasets


In [4]:
def respace_to_0_4(path_file):
    image = sitk.ReadImage(path_file)
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator = sitk.sitkLinear
    resample.SetOutputDirection = image.GetDirection()
    resample.SetOutputOrigin = image.GetOrigin()
    new_spacing = np.array((0.4000000059604645, 0.4000000059604645, 0.4000000059604645))
    resample.SetOutputSpacing(new_spacing)
    orig_size = np.array(image.GetSize(), dtype=int)
    orig_spacing = image.GetSpacing()
    new_size = orig_size * (orig_spacing / new_spacing)
    new_size = np.ceil(new_size).astype(int)  #  Image dimensions are in integers
    new_size = [int(s) for s in new_size]
    resample.SetSize(new_size)
    newimage = resample.Execute(image)
    return newimage




def resample_simg(
    itkimage: sitk.Image,
    newSpacing=(0.4000000059604645, 0.4000000059604645, 0.4000000059604645),
    label=False,
) -> sitk.Image:
    """
    image resize withe sitk resampleImageFilter
    :param itkimage:
    :param newSpacing:such as [1,1,1]
    :param resamplemethod:
    :return:
    """
    newSpacing = np.array(newSpacing, float)
    originSpcaing = itkimage.GetSpacing()
    resampler = sitk.ResampleImageFilter()
    originSize = itkimage.GetSize()
    factor = newSpacing / originSpcaing
    newSize = originSize / factor
    newSize = newSize.astype(int)
    resampler.SetReferenceImage(itkimage)
    resampler.SetOutputSpacing(newSpacing.tolist())
    resampler.SetSize(newSize.tolist())
    resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))
    if label:
        resamplemethod = sitk.sitkNearestNeighbor
    else:
        resamplemethod = sitk.sitkLinear
    resampler.SetInterpolator(resamplemethod)
    itkimgResampled = resampler.Execute(itkimage)
    return itkimgResampled

In [5]:
val2_im_dir = Path("/home/ceballosarroyo.a/workspace/datasets/aneurysm/external")
val2_mask_dir = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/external_label"
)

val2_im_dest = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/external_0.4_crop"
)
val2_mask_dest = Path(
    "/home/ceballosarroyo.a/workspace/datasets/aneurysm/external_label_0.4_crop"
)

val2_im_files = sorted(list(glob(f"{val2_im_dir}/*")))
val2_mask_files = sorted(list(glob(f"{val2_mask_dir}/*")))
len(val2_im_files)

138

In [7]:
combs = [
    ("val2", "label"),
]  # ("val2", "scan")]
crop_threshold_test = 425

In [9]:
offsets = []
for part, ver in combs:
    print(f"Doing {part}, {ver}")
    # define where to put files
    if ver == "scan":
        files = val2_im_files
        dest_dir = val2_im_dest
        crop_threshold = crop_threshold_test
        label = False
        # path_offsets = path_offsets_test_label
    elif ver == "label":
        files = val2_mask_files
        dest_dir = val2_mask_dest
        crop_threshold = crop_threshold_test
        label = True
        # path_offsets = path_offsets_test_scan
    else:
        raise ValueError("ver must be 'scan' or 'label'")
    dict_crop_offsets = {}
    for idx in range(0, 2 + 1):  # len(files)):
        print(idx)
        scan = sitk.ReadImage(files[idx])
        scan = resample_simg(scan, label=label)
        array = sitk.GetArrayFromImage(scan)
        scan_seriesuid = files[idx].split("/")[-1]
        if array.shape[0] > crop_threshold:
            crop_point = array.shape[0] - crop_threshold
            # new_array = array[crop_point:, :, :]
            crop = sitk.CropImageFilter()
            crop.SetLowerBoundaryCropSize([0, 0, crop_point])
            cropped_scan = crop.Execute(scan)
            dict_crop_offsets[scan_seriesuid] = crop_point
        else:
            cropped_scan = scan
            dict_crop_offsets[scan_seriesuid] = 0

        # save scan

        path_cropped_scan = dest_dir / scan_seriesuid
        sitk.WriteImage(cropped_scan, path_cropped_scan)
    print("Writing offsets...\n\n\n")
    offsets.append(dict_crop_offsets)

Doing val2, label
0
1
2
Writing offsets...





In [ ]:
# with open(
#         path_offsets,
#         "w",
#     ) as f:
#         json.dump(dict_crop_offsets, f)

# Metadata file

In [ ]:
# make json
meta = {}

path_files = Path(val2_mask_dest)
for file in tqdm.tqdm(path_files.glob("*.nii.gz")):
    header = sitk.ReadImage(file)
    idx = os.path.split(file)[1]
    meta[idx] = {
        "origin": list(header.GetOrigin()),
        "spacing": list(header.GetSpacing()),
    }
    print(header.GetSize())
    print(meta[idx])

# path

path_json = (
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/external_crop_meta.json"
)
with open(path_json, "w") as f:
    json.dump(meta, f)

# Hospital metadata

In [12]:
path_private = Path("/work/vig/Datasets/aneurysm/hospital/crop_0.4_label")

In [3]:
gt = np.array([12.000001817941666, -28.129431530833244, -11.449998989701271])
# pred = np.array(
#     [
#         306.2348,
#         318.88156,
#         172.51349,
#     ]
# )

gt = np.array([-4.985154554247856, -293.51484574377537, 1385.60002528131])

pred = 283.64624, 332.81552, 144.9039

pred = (
    pred
    * np.array(
        [
            0.4000000059604645,
            0.4000000059604645,
            0.4000000059604645,
        ]
    )
    + np.array([-118.78515625, -159.71484375, 1327.4000244140625])
    * np.array([1, -1, 1])
) * np.array([1, -1, 1])

print("Prediction after respacing\n", pred)
print("GT\n", gt)

Prediction after respacing
 [  -5.32665856  -26.58863377 1385.36158528]
GT
 [  -4.98515455 -293.51484574 1385.60002528]


In [ ]:
def fix_bad_origin(pred, spacing, origin):
    pred_part =  pred*spacing + origin*np.array([1, -1, 1])
    return pred_part*np.array([1, -1, 1])

In [21]:
path_private = Path("/work/vig/Datasets/aneurysm/hospital/crop_0.4_label")
meta = {}
for file in tqdm.tqdm(path_private.glob("*.nii.gz")):

    header = sitk.ReadImage(str(file))
    idx = os.path.split(file)[1]
    meta[idx] = {
        "origin": list(header.GetOrigin()),
        "spacing": list(header.GetSpacing()),
    }
    print(header.GetSize())
    print(meta[idx])
    print(idx)

# path

path_json = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/metadata/hospital_meta.json"
with open(path_json, "w") as f:
    json.dump(meta, f)

1it [00:01,  1.09s/it]

(550, 550, 439)
{'origin': [-110.0, 99.67057037353516, -79.25], 'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]}
CA_00000_0000.nii.gz


2it [00:02,  1.04s/it]

(550, 550, 431)
{'origin': [-118.78515625, -159.71484375, 1327.4000244140625], 'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]}
CA_00001_0000.nii.gz


3it [00:03,  1.12s/it]

...

(550, 550, 414)
{'origin': [-110.0, 115.07057189941406, -73.75], 'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]}
CA_00035_0000.nii.gz


37it [00:43,  1.11s/it]

(535, 535, 445)
{'origin': [-112.791015625, -53.708984375, -233.1999969482422], 'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]}
CA_00036_0000.nii.gz


38it [00:45,  1.19s/it]

(550, 550, 816)
{'origin': [-108.4000015258789, 109.07057189941406, -210.375], 'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]}
CA_00037_0000.nii.gz


# Old code


In [ ]:

cutoff = 500
new_array = array[cutoff:]

# please verify this:
# world_coor = old_pixel_coor * scale + old_origin
#            = new_pixel_coor * scale + new_origin
# new_pixel_coor = old_pixel_coor - cutoff
# => new_origin = old_origin + cutoff*scale

new_origin = None
new_scan = sitk.GetImageFromArray(new_array)
new_scan.CopyInformation(scan)
new_scan.SetOrigin(new_origin)

output_path = None
sitk.WriteImage(new_scan, output_path)

special_cases = {
    "Tr0734.nii.gz": 550,
    "Tr1058.nii.gz": 750,
    "Tr1178.nii.gz": 600,
}
df_train_crop[df_train_crop["coordZ"] < 0]

In [10]:
import os

data_dir = "/work/vig/Datasets/aneurysm/internal_train/crop_0.4"
label_dir = "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_label"


def gen_path(data_dir, label_dir):
    image_list = []

    for scan_path in os.listdir(data_dir):
        image_name = scan_path.split("/")[-1]
        label_paths = []
        if "internal" in data_dir:
            label_name = os.path.join(label_dir, scan_path)

        if os.path.exists(label_name):
            label_paths.append(label_name)
        # if os.path.exists(label_fus):
        #     label_paths.append(label_fus)

        image_list.append([scan_path, label_paths])
        break
    return image_list


image_list = gen_path(data_dir, label_dir)
image_list



ModuleNotFoundError: No module named 'sitk'

In [12]:
import SimpleITK as sitk